In [20]:
import os
import re
import json
import nltk
import pandas as pd
from functools import reduce
from nltk.tokenize import word_tokenize
from iteration_utilities import deepflatten
from nltk.util import ngrams


In [33]:
#corpus name
corpus='US2016tv'
# Path of folder containing the corpus
directory = os.fsencode(r'C:/Users/Aymane-JARAH/Desktop/Thesis/'+corpus)
list_files=[]
appended_dataframe = []
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".json"): 
          appended_dataframe.append(create_dataframe(filename,corpus))
     else:
         continue
appended_data = pd.concat(appended_dataframe)

In [22]:
def create_dataframe(filename,corpus):
    with open(r'C:/Users/Aymane-JARAH/Desktop/Thesis/'+corpus+'/'+filename) as f:
        json_node = json.load(f) 
    get_nodes_with_question_type(json_node)
    nodes_check=[d['nodeID'] for d in get_nodes_with_question_type(json_node)]
    questions=[d['fromID'] for d in list(filter(lambda x: (x['toID'] in nodes_check), json_node['edges']))]
    
    question_types=[d['toID'] for d in list(filter(lambda x: (x['fromID'] in questions), json_node['edges']))]
    question_types=list(filter(lambda x: (get_text_from_node_id(x,json_node) in list_), question_types))
    
    data_nodes_id=pd.DataFrame({'Types':[get_text_from_node_id(node_id,json_node) for node_id in question_types],
                            'Questions':[get_text_from_node_id(node_id,json_node) for node_id in questions],
                            'nodeID': [node_id for node_id in questions],
                            'PersonID':[get_text_from_node_id(node_id,json_node).split(':')[0] for node_id in questions],
                            'Precedents':[get_precedents(question,json_node) for question in questions],
                            'Succesors':[get_succesors(question,json_node) for question in questions],
                            'NodeSet':[filename]*len(questions)
                           })
    return data_nodes_id


In [23]:
def get_text_from_node_id(node_id,json_file):
    node = list(filter(lambda x: (x['nodeID'] == node_id), json_file['nodes']))
    if node:
        return node[0]['text']
    else:
        return None

In [24]:
def get_nodes_with_question_type(json_nodes):
    questions_nodes = list(filter(lambda x: (x['text'] in list_), json_nodes['nodes']))
    return questions_nodes




In [25]:
def get_person_from_node_id(node_id,json_file):
    node = list(filter(lambda x: (x['nodeID'] == node_id), json_file['locutions']))
    if node:
        return node[0]['personID']
    else:
        return None


In [26]:
def get_type_from_node_id(node_id,json_file):
    node = list(filter(lambda x: (x['nodeID'] == node_id), json_file['nodes']))
    if node:
        return node[0]['type']
    else:
        return None


In [32]:
def get_precedents(question_id,json_file):
    prec=[]
    questions=list(filter(lambda x: (x['toID']==question_id), json_file['edges']))
    if questions:
        for q in questions:
            
            L_precedents=list(filter(lambda x: (x['toID'] in [q['fromID']]), json_file['edges']))
            L_precedents=list(filter(lambda x: (get_type_from_node_id(x['fromID'],json_file)=='L'), L_precedents))
            
            if L_precedents:
                prec.append(L_precedents)
            
    else:
        prec=[]

    return [{get_text_from_node_id(precedent[0]['fromID'],json_file).split(':')[0]:get_text_from_node_id(precedent[0]['fromID'],json_file)} for precedent in prec]


In [28]:
def get_succesors(question_id,json_file):
    succ=[]
    questions=list(filter(lambda x: (x['fromID']==question_id), json_file['edges']))
    questions=list(filter(lambda x: (get_type_from_node_id(x['toID'],json_file)=='TA'), questions))
    if questions:
        for q in questions:
            succ.append(list(filter(lambda x: (x['fromID'] in [q['toID']]), json_file['edges'])))       
    else:
        succ=[]
    return [{get_text_from_node_id(succesor[0]['toID'],json_file).split(':')[0]:get_text_from_node_id(succesor[0]['toID'],json_file)} for succesor in succ]


In [29]:
appended_data['Succesors']=appended_data['Succesors'].apply(lambda l : [set(e.values()) for e in l])
appended_data['Precedents']=appended_data['Precedents'].apply(lambda l : [set(e.values()) for e in l])
appended_data.to_csv(corpus+'.csv', encoding='utf-8', index=False)